In [1]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

<IPython.core.display.Javascript object>

Домашка по 6му занятию.

Постройте регрессионную модель, используя все те знания, что получили до этого.

Обращаю внимание, что чем подробней вы выполняете задание, тем больше понимания у меня ваших скиллов 🙂

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('house_prices.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
df = df.drop_duplicates()
df.shape

(1460, 81)

In [5]:
na_vals = df.isna().sum()
na_vals[na_vals > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [6]:
df = df.fillna(0)

Многие категориальные значения можно закодировать числами, так как прослеживается строгая иерархия значений:

Ex	Excellent 5

Gd	Good 4

TA	Typical/Average 3

Fa	Fair 2

Po	Poor 1

NA	None 0

In [7]:
object_columns = df.dtypes[df.dtypes == 'object'].index.values

marks = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}
mark_cols = list()

for key, value in marks.items():
    for column in object_columns:
        if (df.loc[df[column] == key, column].shape[0] > 1)&~(column in mark_cols):
            mark_cols.append(column)
        df.loc[df[column] == key, column] = value

In [8]:
for column in mark_cols:
    try:
        df[column] = df[column].astype('int64')
    except:
        pass

Так как MSSubClass -- это на самом деле категориальное значение, закодируем и его в бинарный признак.

In [9]:
object_columns = df.dtypes[df.dtypes == 'object'].index.values
np.append(object_columns, 'MSSubClass')

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'Foundation', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageType',
       'GarageFinish', 'PavedDrive', 'Fence', 'MiscFeature', 'SaleType',
       'SaleCondition', 'MSSubClass'], dtype=object)

In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
ohe = OneHotEncoder()

for column in object_columns:
    try:
        column_ohe = ohe.fit_transform(df[[column]]).toarray()
        column_ohe_df = pd.DataFrame(column_ohe, columns = ohe.categories_[0])
        column_ohe_df.index = df.index
        df = df.join(column_ohe_df)
    except:
        pass

C:\Users\valer\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:395: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  check_array(X, dtype=np.int)
C:\Users\valer\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:110: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current us

In [12]:
df = df.drop(columns=object_columns, axis=1)
df = df.drop_duplicates()
df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,ConLw,New,Oth,WD,Abnorml,AdjLand,Alloca,Family,Normal,Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,4,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,20,80.0,9600,6,8,1976,1976,0.0,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,60,68.0,11250,7,5,2001,2002,162.0,4,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,70,60.0,9550,7,5,1915,1970,0.0,3,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,60,84.0,14260,8,5,2000,2000,350.0,4,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error

C:\Users\valer\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\valer\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

In [14]:
estimator = LinearRegression()

X = df.drop(columns=['SalePrice'], axis=1)
y = df['SalePrice']

In [15]:
estimator.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
estimator.intercept_, estimator.coef_

(-657850.1515167662,
 array([ 7.11310030e-01, -7.98792883e+01,  1.07895178e+01,  4.40687714e-01,
         8.91739403e+03,  6.59749984e+03,  2.73908242e+02, -4.87437609e+01,
         1.99288739e+01,  4.70790312e+03, -1.65990772e+03,  4.16763452e+03,
        -6.47491520e+03,  1.63388789e+01,  4.72763174e+00, -2.38481755e+00,
         1.86817008e+01,  5.68793828e+02,  1.62819920e+01,  3.08002746e+01,
        -2.11646303e+01,  2.59175550e+01,  3.02273441e+03,  9.90249075e+02,
         1.10068301e+03,  1.91827270e+03, -5.79728738e+03, -2.05949650e+04,
         7.58542045e+03,  3.69586189e+03,  7.80619793e+03, -1.78389349e+03,
        -1.16843968e+01,  6.82247620e+03,  1.13855274e+01,  6.36341060e+03,
        -3.60240951e+03,  1.53780626e+01,  1.21228966e+01, -5.08086784e+00,
         3.95069057e+01,  3.93100788e+01, -1.38573389e+02,  3.74067902e+04,
        -1.01350425e+00, -7.32651296e+02,  3.69485229e+01, -1.69104067e+04,
         7.00635688e+03,  5.30301399e+03,  3.12698841e+03,  1.47404

In [17]:
y_predicted = estimator.predict(X)

print(f'MAE: {mean_absolute_error(y, y_predicted)}')
print(f'MSE: {mean_squared_error(y, y_predicted)}')
print(f'RMSE: {mean_squared_error(y, y_predicted) ** 0.5}')

MAE: 15976.21195701058
MSE: 605426089.4955678
RMSE: 24605.40772869996
